<a href="https://colab.research.google.com/github/Lima-Ricardo/Python_Webscraping/blob/main/Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv

#Intalando e configurando o webdriver do firefo
att = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=att)

#Passo 01: Fazendo login no linkedin

#Abindo o Firfox e fazendo longin no linkedin

#Definindo a URL
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(3)

#campo de email
login = open('C:\\Users\\rilim_rcalv72\\PycharmProjects\\pythonProject\\credenciais.txt')
line = login.readlines()
email = line[0]
senha = line[1]
field_email = driver.find_element('id','username')
field_email.send_keys(email)
sleep(2)

#campo de senha
field_password = driver.find_element('name', 'session_password')
field_password.send_keys(senha)
sleep(3)

#botão entrar
button_enter = driver.find_element('xpath','/html/body/div/main/div[2]/div[1]/form/div[3]/button')
button_enter.click()
sleep(2)

#Passo 02 usando o campo de busca

#Localizando o campo de busca
search_field = driver.find_element('xpath','/html/body/div[5]/header/div/div/div/div[1]/input')

# 2.2 Inserindo os dados no campo de busca
search_query = input('Qual perfil você quer fazer o scrape? ')
search_field.send_keys(search_query)

# 2.3: Trazendo os resultados
search_field.send_keys(Keys.RETURN)


#Passo 03 fazendo o webscraping


# 3.1 Nanegando na url
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link') #('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        # profile_ID = profile.get('href')
        # profile_URL = "https://www.linkedin.com" + profile_ID
        profile_URL = profile.get('href')
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# 3.2 navegando de página em página e extraindo as informações

input_page = int(input('Em quantas páginas você quer fazer o scrape?: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(3)

print('- Iniciando a gravação dos dados')


#Passo 4 coletar e armazenar os resultados

# Fazendo a coleta dos perfis e gravando em um aqruivo .CSV

with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Nome', 'Profissão', 'Localização', 'URL_Perfil']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)
        sleep(3)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        try:
            name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters
            print('--- Profile name is: ', name)
            location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters
            print('--- Profile location is: ', location)
            title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
            print('--- Profile title is: ', title)
            writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
            print('\n')
        except:
            pass

print('Finalizado!')